<a href="https://colab.research.google.com/github/zhang8yiming/ML-DL/blob/main/L7_Homework_Reference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 用不同的优化方法训练验证码识别模型

## 作业内容

在本次作业中，你将：
- 用不同的优化方法训练验证码识别模型；
- 调整学习率提高模型精度；
- 查看神经网络识别验证码字母的效果。

## 数据集介绍

数据集是验证码图像，图像是 **5 个字母** 的单词，并且含有噪点（模糊和线条）。 它们的尺寸为 **200 x 50**。 文件名与图像字母相同。数据集来源 [kaggle](https://www.kaggle.com/fournierp/captcha-version-2-images).

In [ ]:
import os
from torch.utils.data import DataLoader,Dataset
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import pandas as pd
from torchvision import models
import torch.nn as nn
import torch
from torch.autograd import Variable

In [ ]:
NUMBER = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
ALPHABET = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
ALL_CHAR_SET = NUMBER + ALPHABET
ALL_CHAR_SET_LEN = len(ALL_CHAR_SET)
MAX_CAPTCHA = 5

In [ ]:
def encode(a):
    onehot = [0]*ALL_CHAR_SET_LEN
    idx = ALL_CHAR_SET.index(a)
    onehot[idx] += 1
    return onehot

class Mydataset(Dataset):
    def __init__(self, path, transform=None):
        self.path = path
        self.transform = transform
        
    def __getitem__(self, idx):
        img_path = self.path[idx]
        img = Image.open(img_path)
        img = img.convert('L')
        label = img_path[-9:-4]
        # print(img_path, label)
        label_oh = []
        for i in label:
            label_oh += encode(i)
        if self.transform is not None:
            img = self.transform(img)
        return img, np.array(label_oh), label
    
    def __len__(self):
        return len(self.path)

In [ ]:
transform = transforms.Compose([
    transforms.Resize([224, 224]),
    transforms.ToTensor(),
])

In [ ]:
import glob
path = './captcha_images_v2/*.png'

In [ ]:
train_ds = Mydataset(glob.glob(path)[:-100], transform=transform)
test_ds = Mydataset(glob.glob(path)[-100:], transform)
train_dl = DataLoader(train_ds, batch_size=64, num_workers=0)
test_dl = DataLoader(train_ds, batch_size=1, num_workers=0)

In [ ]:
train_ds[0]

(tensor([[[0.7529, 0.7529, 0.7529,  ..., 0.9843, 0.9843, 0.9843],
          [0.7529, 0.7529, 0.7529,  ..., 0.9843, 0.9843, 0.9843],
          [0.7529, 0.7529, 0.7529,  ..., 0.9843, 0.9843, 0.9843],
          ...,
          [0.7647, 0.7647, 0.7647,  ..., 0.9961, 0.9961, 0.9961],
          [0.7647, 0.7647, 0.7647,  ..., 0.9961, 0.9961, 0.9961],
          [0.7647, 0.7647, 0.7647,  ..., 0.9961, 0.9961, 0.9961]]]),
 array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
model = models.resnet18(pretrained=True)
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = nn.Linear(in_features=512, out_features=ALL_CHAR_SET_LEN*MAX_CAPTCHA, bias=True)

In [ ]:
loss_func = nn.MultiLabelSoftMarginLoss()
optm = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
for epoch in range(3):
    for step, i in enumerate(train_dl):
        img, label_oh, label = i
        pred = model(img)
        loss = loss_func(pred, label_oh.float())
        optm.zero_grad()
        loss.backward()
        optm.step()
        print('eopch:', epoch+1, 'step:', step+1, 'loss:', loss.item())

eopch: 1 step: 1 loss: 0.7659690380096436
eopch: 1 step: 2 loss: 0.5586223006248474
eopch: 1 step: 3 loss: 0.3903140723705292
eopch: 1 step: 4 loss: 0.2698208689689636
eopch: 1 step: 5 loss: 0.19641001522541046
eopch: 1 step: 6 loss: 0.15496386587619781
eopch: 1 step: 7 loss: 0.13451442122459412
eopch: 1 step: 8 loss: 0.12765008211135864
eopch: 1 step: 9 loss: 0.12417804449796677
eopch: 1 step: 10 loss: 0.12543776631355286
eopch: 1 step: 11 loss: 0.12637782096862793
eopch: 1 step: 12 loss: 0.12565430998802185
eopch: 1 step: 13 loss: 0.1258484423160553
eopch: 1 step: 14 loss: 0.1262722909450531
eopch: 1 step: 15 loss: 0.12370920926332474
eopch: 2 step: 1 loss: 0.1229570060968399
eopch: 2 step: 2 loss: 0.12030627578496933
eopch: 2 step: 3 loss: 0.12015184760093689
eopch: 2 step: 4 loss: 0.11753649264574051
eopch: 2 step: 5 loss: 0.11459614336490631
eopch: 2 step: 6 loss: 0.11422881484031677
eopch: 2 step: 7 loss: 0.11289031058549881
eopch: 2 step: 8 loss: 0.11080260574817657
eopch: 2 ste

In [ ]:
model.eval();

for step, (img, label_oh, label) in enumerate(test_dl):
    pred = model(img)

    c0 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[0:ALL_CHAR_SET_LEN])]
    c1 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN:ALL_CHAR_SET_LEN*2])]
    c2 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*2:ALL_CHAR_SET_LEN*3])]
    c3 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*3:ALL_CHAR_SET_LEN*4])]
    c4 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*4:ALL_CHAR_SET_LEN*5])]
    c = '%s%s%s%s%s' % (c0, c1, c2, c3, c4)

    print('label:', label[0], 'pred:', c)
    
    break

label: 5nggg pred: 37pgg


## 问题1：准确率计算

在上述验证集的代码中并没有包含准确率计算逻辑，请加入以下代码：
- 字符准确率
- 整张图片预测准确率

In [ ]:
model.eval();

char_acc = 0
image_acc = 0
for step, (img, label_oh, label) in enumerate(test_dl):
    pred = model(img)

    c0 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[0:ALL_CHAR_SET_LEN])]
    c1 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN:ALL_CHAR_SET_LEN*2])]
    c2 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*2:ALL_CHAR_SET_LEN*3])]
    c3 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*3:ALL_CHAR_SET_LEN*4])]
    c4 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*4:ALL_CHAR_SET_LEN*5])]
    c = '%s%s%s%s%s' % (c0, c1, c2, c3, c4)

    for c1, c2 in zip(label[0], c):
        if c1 == c2:
            char_acc += 1
    
    if label[0] == c:
        image_acc += 1
    
char_acc = char_acc / len(test_dl.dataset) / 5
image_acc = image_acc / len(test_dl.dataset)

In [ ]:
char_acc, image_acc
# 训练时间较短，整张图片预测准确率较低正常

(0.3087234042553192, 0.0)

## 问题2：优化器尝试

选择四种优化器，分别训练5个epoch，然后记录下各自在测试集精度。

### SGD

In [ ]:
model = models.resnet18(pretrained=True)
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = nn.Linear(in_features=512, out_features=ALL_CHAR_SET_LEN*MAX_CAPTCHA, bias=True)

loss_func = nn.MultiLabelSoftMarginLoss()
optm = torch.optim.SGD(model.parameters(), lr=0.001)

for epoch in range(5):
    for step, i in enumerate(train_dl):
        img, label_oh, label = i
        pred = model(img)
        loss = loss_func(pred, label_oh.float())
        optm.zero_grad()
        loss.backward()
        optm.step()

model.eval();
char_acc = 0
image_acc = 0
for step, (img, label_oh, label) in enumerate(test_dl):
    pred = model(img)

    c0 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[0:ALL_CHAR_SET_LEN])]
    c1 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN:ALL_CHAR_SET_LEN*2])]
    c2 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*2:ALL_CHAR_SET_LEN*3])]
    c3 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*3:ALL_CHAR_SET_LEN*4])]
    c4 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*4:ALL_CHAR_SET_LEN*5])]
    c = '%s%s%s%s%s' % (c0, c1, c2, c3, c4)

    for c1, c2 in zip(label[0], c):
        if c1 == c2:
            char_acc += 1
    
    if label[0] == c:
        image_acc += 1
    
char_acc = char_acc / len(test_dl.dataset) / 5
image_acc = image_acc / len(test_dl.dataset)
print(char_acc, image_acc)

0.0225531914893617 0.0


### AdamW

In [ ]:
model = models.resnet18(pretrained=True)
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = nn.Linear(in_features=512, out_features=ALL_CHAR_SET_LEN*MAX_CAPTCHA, bias=True)

loss_func = nn.MultiLabelSoftMarginLoss()
optm = torch.optim.AdamW(model.parameters(), lr=0.001)

for epoch in range(5):
    for step, i in enumerate(train_dl):
        img, label_oh, label = i
        pred = model(img)
        loss = loss_func(pred, label_oh.float())
        optm.zero_grad()
        loss.backward()
        optm.step()

model.eval();
char_acc = 0
image_acc = 0
for step, (img, label_oh, label) in enumerate(test_dl):
    pred = model(img)

    c0 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[0:ALL_CHAR_SET_LEN])]
    c1 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN:ALL_CHAR_SET_LEN*2])]
    c2 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*2:ALL_CHAR_SET_LEN*3])]
    c3 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*3:ALL_CHAR_SET_LEN*4])]
    c4 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*4:ALL_CHAR_SET_LEN*5])]
    c = '%s%s%s%s%s' % (c0, c1, c2, c3, c4)

    for c1, c2 in zip(label[0], c):
        if c1 == c2:
            char_acc += 1
    
    if label[0] == c:
        image_acc += 1
    
char_acc = char_acc / len(test_dl.dataset) / 5
image_acc = image_acc / len(test_dl.dataset)
print(char_acc, image_acc)

0.7523404255319149 0.2351063829787234


### Adadelta

In [ ]:
model = models.resnet18(pretrained=True)
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = nn.Linear(in_features=512, out_features=ALL_CHAR_SET_LEN*MAX_CAPTCHA, bias=True)

loss_func = nn.MultiLabelSoftMarginLoss()
optm = torch.optim.Adadelta(model.parameters(), lr=0.001)

for epoch in range(5):
    for step, i in enumerate(train_dl):
        img, label_oh, label = i
        pred = model(img)
        loss = loss_func(pred, label_oh.float())
        optm.zero_grad()
        loss.backward()
        optm.step()

model.eval();
char_acc = 0
image_acc = 0
for step, (img, label_oh, label) in enumerate(test_dl):
    pred = model(img)

    c0 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[0:ALL_CHAR_SET_LEN])]
    c1 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN:ALL_CHAR_SET_LEN*2])]
    c2 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*2:ALL_CHAR_SET_LEN*3])]
    c3 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*3:ALL_CHAR_SET_LEN*4])]
    c4 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*4:ALL_CHAR_SET_LEN*5])]
    c = '%s%s%s%s%s' % (c0, c1, c2, c3, c4)

    for c1, c2 in zip(label[0], c):
        if c1 == c2:
            char_acc += 1
    
    if label[0] == c:
        image_acc += 1
    
char_acc = char_acc / len(test_dl.dataset) / 5
image_acc = image_acc / len(test_dl.dataset)
print(char_acc, image_acc)

/home/lyz/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/lyz/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


0.02191489361702128 0.0


### Adagrad

In [ ]:
model = models.resnet18(pretrained=True)
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = nn.Linear(in_features=512, out_features=ALL_CHAR_SET_LEN*MAX_CAPTCHA, bias=True)

loss_func = nn.MultiLabelSoftMarginLoss()
optm = torch.optim.Adagrad(model.parameters(), lr=0.001)

for epoch in range(5):
    for step, i in enumerate(train_dl):
        img, label_oh, label = i
        pred = model(img)
        loss = loss_func(pred, label_oh.float())
        optm.zero_grad()
        loss.backward()
        optm.step()

model.eval();
char_acc = 0
image_acc = 0
for step, (img, label_oh, label) in enumerate(test_dl):
    pred = model(img)

    c0 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[0:ALL_CHAR_SET_LEN])]
    c1 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN:ALL_CHAR_SET_LEN*2])]
    c2 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*2:ALL_CHAR_SET_LEN*3])]
    c3 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*3:ALL_CHAR_SET_LEN*4])]
    c4 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*4:ALL_CHAR_SET_LEN*5])]
    c = '%s%s%s%s%s' % (c0, c1, c2, c3, c4)

    for c1, c2 in zip(label[0], c):
        if c1 == c2:
            char_acc += 1
    
    if label[0] == c:
        image_acc += 1
    
char_acc = char_acc / len(test_dl.dataset) / 5
image_acc = image_acc / len(test_dl.dataset)
print(char_acc, image_acc)

/home/lyz/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/lyz/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


0.6861702127659575 0.1478723404255319


### RMSprop

In [ ]:
model = models.resnet18(pretrained=True)
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = nn.Linear(in_features=512, out_features=ALL_CHAR_SET_LEN*MAX_CAPTCHA, bias=True)

loss_func = nn.MultiLabelSoftMarginLoss()
optm = torch.optim.RMSprop(model.parameters(), lr=0.001)

for epoch in range(5):
    for step, i in enumerate(train_dl):
        img, label_oh, label = i
        pred = model(img)
        loss = loss_func(pred, label_oh.float())
        optm.zero_grad()
        loss.backward()
        optm.step()

model.eval();
char_acc = 0
image_acc = 0
for step, (img, label_oh, label) in enumerate(test_dl):
    pred = model(img)

    c0 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[0:ALL_CHAR_SET_LEN])]
    c1 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN:ALL_CHAR_SET_LEN*2])]
    c2 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*2:ALL_CHAR_SET_LEN*3])]
    c3 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*3:ALL_CHAR_SET_LEN*4])]
    c4 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*4:ALL_CHAR_SET_LEN*5])]
    c = '%s%s%s%s%s' % (c0, c1, c2, c3, c4)

    for c1, c2 in zip(label[0], c):
        if c1 == c2:
            char_acc += 1
    
    if label[0] == c:
        image_acc += 1
    
char_acc = char_acc / len(test_dl.dataset) / 5
image_acc = image_acc / len(test_dl.dataset)
print(char_acc, image_acc)

## 问题3：学习率调整

在选择最优的优化器中分别加入2中学习率调整策略，然后记录下各自在测试集精度。

### AdamW + StepLR

In [ ]:
model = models.resnet18(pretrained=True)
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = nn.Linear(in_features=512, out_features=ALL_CHAR_SET_LEN*MAX_CAPTCHA, bias=True)

loss_func = nn.MultiLabelSoftMarginLoss()
optm = torch.optim.AdamW(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optm, step_size=50, gamma=0.7)

for epoch in range(5):
    for step, i in enumerate(train_dl):
        img, label_oh, label = i
        pred = model(img)
        loss = loss_func(pred, label_oh.float())
        optm.zero_grad()
        loss.backward()
        optm.step()
        scheduler.step()

model.eval();
char_acc = 0
image_acc = 0
for step, (img, label_oh, label) in enumerate(test_dl):
    pred = model(img)

    c0 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[0:ALL_CHAR_SET_LEN])]
    c1 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN:ALL_CHAR_SET_LEN*2])]
    c2 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*2:ALL_CHAR_SET_LEN*3])]
    c3 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*3:ALL_CHAR_SET_LEN*4])]
    c4 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*4:ALL_CHAR_SET_LEN*5])]
    c = '%s%s%s%s%s' % (c0, c1, c2, c3, c4)

    for c1, c2 in zip(label[0], c):
        if c1 == c2:
            char_acc += 1
    
    if label[0] == c:
        image_acc += 1
    
char_acc = char_acc / len(test_dl.dataset) / 5
image_acc = image_acc / len(test_dl.dataset)
print(char_acc, image_acc)

0.8176595744680851 0.3478723404255319


### AdamW + CosineAnnealingLR

In [ ]:
model = models.resnet18(pretrained=True)
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = nn.Linear(in_features=512, out_features=ALL_CHAR_SET_LEN*MAX_CAPTCHA, bias=True)

loss_func = nn.MultiLabelSoftMarginLoss()
optm = torch.optim.AdamW(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optm, T_max=100, eta_min=0)

for epoch in range(5):
    for step, i in enumerate(train_dl):
        img, label_oh, label = i
        pred = model(img)
        loss = loss_func(pred, label_oh.float())
        optm.zero_grad()
        loss.backward()
        optm.step()
        scheduler.step()

model.eval();
char_acc = 0
image_acc = 0
for step, (img, label_oh, label) in enumerate(test_dl):
    pred = model(img)

    c0 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[0:ALL_CHAR_SET_LEN])]
    c1 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN:ALL_CHAR_SET_LEN*2])]
    c2 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*2:ALL_CHAR_SET_LEN*3])]
    c3 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*3:ALL_CHAR_SET_LEN*4])]
    c4 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*4:ALL_CHAR_SET_LEN*5])]
    c = '%s%s%s%s%s' % (c0, c1, c2, c3, c4)

    for c1, c2 in zip(label[0], c):
        if c1 == c2:
            char_acc += 1
    
    if label[0] == c:
        image_acc += 1
    
char_acc = char_acc / len(test_dl.dataset) / 5
image_acc = image_acc / len(test_dl.dataset)
print(char_acc, image_acc)

0.6568085106382979 0.09574468085106383
